In [1]:
import os
import sys

def import_package(depth: int, is_jupyter: bool):
    if is_jupyter is False:
        current_dir = os.path.dirname(os.path.abspath(__file__))
    else:
        current_dir = os.getcwd()

    parent_dir = current_dir
    while depth > 0:
        depth -= 1
        parent_dir = os.path.dirname(parent_dir)
        
    sys.path.insert(0, parent_dir)

import_package(1, True)


In [2]:
from src.labeling.ETLProcessor import ETLProcessor
from src.labeling.utils import get_spark, query
from omegaconf import OmegaConf

In [3]:
config = OmegaConf.load("../config_small.yml")
spark = get_spark('../lib/postgresql-42.5.0.jar')

df = query(spark, **config.db)
df.printSchema()

22/12/09 13:19:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Successfully queried data from database
root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: boolean (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: float (nullable = true)
 |-- delegator_shares: decimal(38,0) (nullable = true)
 |-- self_bonded: decimal(38,0) (nullable = true)
 |-- propose: boolean (nullable = true)
 |-- vote: boolean (nullable = true)



In [4]:
accept_rate = 0.1
concentration_level = 0.9
vote_score = 2
propose_score = 6
A = 9
B = 4
C = 2
D = 4
"""
suitable parameters are:
vote_proposed_win_size = 14400 (blocks) 
combine_win_size = 150 (blocks)
label_win_size = 100800 (blocks)
Approximate each block ~~ 6seconds
"""
vote_proposed_win_size = 500 #This is just test, favourable is
combine_win_size = 20 
label_win_size = 60


df = ETLProcessor.data_scoring(df, accept_rate, concentration_level, vote_score, propose_score, A, B, C, D,vote_proposed_win_size, combine_win_size, label_win_size)


------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
------------------------------------------------
Sucessfully combine data
------------------------------------------------
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [5]:
df.count()

756

In [6]:
df.show(800)

+--------------------+--------------------+------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------------------+------------------+
|    operator_address|     comission_score|             score|voting_power_score|tokens_proportion|     commission_rate|         self_bonded|   self_bonded_score|    delegator_shares|           tokens|block_height|vote_propose_score|         new_score|
+--------------------+--------------------+------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------------------+------------------+
|oraivaloper1u2344...|  0.6999999918230734| 6.047454316199156|               0.0|     0.0327370000|0.030000000817692673|32510000000000000...| 0.26053762266784203|26308457537000000...| 26308457537.0000|     7049971|0.6815947758927939| 6.04745

In [7]:
df.printSchema()

root
 |-- operator_address: string (nullable = true)
 |-- comission_score: double (nullable = true)
 |-- score: double (nullable = true)
 |-- voting_power_score: double (nullable = true)
 |-- tokens_proportion: decimal(38,10) (nullable = true)
 |-- commission_rate: double (nullable = true)
 |-- self_bonded: decimal(38,4) (nullable = true)
 |-- self_bonded_score: double (nullable = true)
 |-- delegator_shares: decimal(38,4) (nullable = true)
 |-- tokens: decimal(38,4) (nullable = true)
 |-- block_height: integer (nullable = true)
 |-- vote_propose_score: double (nullable = true)
 |-- new_score: double (nullable = true)

